In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Swarnendu\\Desktop\\End-to-end-Chest-Cancer-Classification-using-MLflow-DVC\\research'

In [3]:
import sys
sys.path.append('C:/Users/Swarnendu/Desktop/End-to-end-Chest-Cancer-Classification-using-MLflow-DVC/src')
print(sys.executable)

c:\Users\Swarnendu\anaconda3\envs\cancer\python.exe


In [4]:
import os

os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/banik-s/End-to-end-Chest-Cancer-Classification-using-MLflow-DVC.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "banik-s"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "fb6fbf399f88cb7bee84d680257e90ebe568cc13"


In [16]:
import os
from pathlib import Path
import tensorflow as tf

# Set the working directory to where your research folder is located
os.chdir(r"C:\Users\Swarnendu\Desktop\End-to-end-Chest-Cancer-Classification-using-MLflow-DVC")

# Get the current working directory
base_dir = Path.cwd()

# Define the relative path to your model file
model_path = base_dir / "research" / "artifacts" / "training" / "model.h5"

# Print the path to verify
print(f"Model path: {model_path}")

# Load the model using the relative path
model = tf.keras.models.load_model(str(model_path))


Model path: C:\Users\Swarnendu\Desktop\End-to-end-Chest-Cancer-Classification-using-MLflow-DVC\research\artifacts\training\model.h5


In [6]:
import sys
sys.path.append('C:/Users/Swarnendu/Desktop/End-to-end-Chest-Cancer-Classification-using-MLflow-DVC/src')
print(sys.executable)

c:\Users\Swarnendu\anaconda3\envs\cancer\python.exe


In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [8]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [26]:
# Define your ConfigurationManager class
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self) -> EvaluationConfig:
        # Define the base directory where the script is executed
        base_dir = Path.cwd()

        # Define the relative path to your model file and data directory
        model_path = base_dir / "research" / "artifacts" / "training" / "model.h5"
        training_data_path = base_dir / "research" / "extracted_files" / "Chest-CT-Scan-data"

        # Ensure paths exist
        if not model_path.exists():
            raise FileNotFoundError(f"Model file not found: {model_path}")
        if not training_data_path.exists():
            raise FileNotFoundError(f"Training data directory not found: {training_data_path}")

        eval_config = EvaluationConfig(
            path_of_model=str(model_path),
            training_data=str(training_data_path),
            mlflow_uri="https://dagshub.com/banik-s/End-to-end-Chest-Cancer-Classification-using-MLflow-DVC.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [27]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [28]:
# Define your Evaluation class
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _valid_generator(self):
        datagenerator_kwargs = dict(
            rescale=1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)

    def evaluation(self):
        self.model = self.load_model(Path(self.config.path_of_model))
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics({"loss": self.score[0], "accuracy": self.score[1]})
            
            if tracking_url_type_store != "file":
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [30]:
# Main execution block
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()
except Exception as e:
    print(f"An error occurred: {e}")
    raise e

[2024-08-16 19:39:06,409: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-08-16 19:39:06,419: INFO: common: yaml file: params.yaml loaded successfully]
[2024-08-16 19:39:06,423: INFO: common: created directory at: artifacts]
Found 102 images belonging to 2 classes.
7/7 [==============================] - 48s 7s/step - loss: 0.7053 - accuracy: 0.4314
[2024-08-16 19:39:56,428: INFO: common: json file saved at: scores.json]


2024/08/16 19:40:02 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\SWARNE~1\AppData\Local\Temp\tmps11kty2n\model\data\model\assets
[2024-08-16 19:40:07,159: INFO: builder_impl: Assets written to: C:\Users\SWARNE~1\AppData\Local\Temp\tmps11kty2n\model\data\model\assets]


2024/08/16 19:40:23 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\SWARNE~1\AppData\Local\Temp\tmps11kty2n\model, flavor: tensorflow). Fall back to return ['tensorflow==2.13.0', 'cloudpickle==3.0.0']. Set logging level to DEBUG to see the full traceback. 
Successfully registered model 'VGG16Model'.
2024/08/16 19:44:34 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: VGG16Model, version 1
Created version '1' of model 'VGG16Model'.
2024/08/16 19:44:34 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2024/08/16 19:44:35 INFO mlflow.tracking._tracking_service.client: 🏃 View run intrigued-pig-499 at: https://dagshub.com/banik-s/End-to-end-Chest-Cancer-Classification-using-MLflow-DVC.mlflow/#/experiments/0/runs/c5e9035265d5402ab4e107cd6dc77617.
2